In [2]:
#file to scrape
from bs4 import BeautifulSoup

#files to login
from lxml import html
import lxml.html
import requests

import pandas as pd

import re
from robobrowser import RoboBrowser

In [56]:
#used to log user into Grade
def getAutoGraderSession(loginURL, Email, Password):
    s = requests.session()
    login = s.get(loginURL)
    login_html = lxml.html.fromstring(login.text)
    hidden_inputs = login_html.xpath(r'//form//input[@type="hidden"]')
    form= {x.attrib["name"]: x.attrib["value"] for x in hidden_inputs}
    #print(form)
    form['username'] = Email
    form['password'] = Password 
    response = s.post(loginURL, data=form)
    response.url
    return (response, s)
    #returns the urls response and the session

In [86]:
(myAutoGrader, session) = getAutoGraderSession('https://autograder.ucsd.edu/login', 'USERNAME@ucsd.edu', 'PASSWORD')
souppy = BeautifulSoup(myAutoGrader.content, 'html.parser')
print(type(session))


<class 'requests.sessions.Session'>


In [87]:
tempYo = souppy.find_all('a')

queue = ''
        
def findQueue(stringValue):
    for i in stringValue:
    
        if 'queue' in i['href']:
            return i
    return ''

queue = findQueue(tempYo)
print(queue['href'])

/queue/248


In [88]:
queueRequest = session.get('https://autograder.ucsd.edu' + queue['href'] + '#pending-tab')


In [89]:
queueSoup = BeautifulSoup(queueRequest.content, 'html.parser')

In [91]:
queueSoup.find('div', {'id': 'news-feed-tab-container'})

<div class="news-feed tab-content" id="news-feed-tab-container">
<div class="tab-pane active" id="news-feed-tab">
</div>
<div class="tab-pane" id="news-archive-tab">
</div>
</div>

In [63]:
#print(souppy.prettify())
#TRYING TO FIND THE FRICKEN LIST FOR THE QUEUE
chicken = souppy.find_all('a')
history = ''
for i in chicken:
    if 'history' in i['href'] and 'queue' in i['href'] :
        history = i['href']
        
print(history)

requestHistory = session.get('https://autograder.ucsd.edu' + history)
prettyHistory = BeautifulSoup(requestHistory.content, 'html.parser')
print(prettyHistory)

/queue/248/history/all
<!DOCTYPE HTML>

<html>
<head>
<title>Ticket History - Autograder</title>
<meta charset="utf-8"/>
<meta content="true" name="HandheldFriendly">
<meta content="width=device-width, minimum-scale=.85, initial-scale=1.0, maximum-scale=1.0, user-scalable=0" name="viewport"/>
<link href="/public/images/favicon.ico" rel="icon" type="image/x-icon"/>
<link href="/public/images/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<link charset="utf-8" href="/public/stylesheets/bootstrap.min.css" rel="stylesheet" type="text/css"/>
<link href="https://gitcdn.github.io/bootstrap-toggle/2.2.2/css/bootstrap-toggle.min.css" rel="stylesheet"/>
<link charset="utf-8" href="/public/stylesheets/bootstrap-nav.css" rel="stylesheet" type="text/css"/>
<link charset="utf-8" href="/public/stylesheets/bootstrap-override.css" rel="stylesheet" type="text/css"/>
<link charset="utf-8" href="/public/stylesheets/font-awesome.min.css" rel="stylesheet" type="text/css"/>
<link charset="utf-8" href

In [40]:
#list of courses currently enrolled
courses = souppy.find_all('option')
currentCourses = []
print(courses)

[<option selected="" title="CSE140L - SPR18 - Digital Systems Laboratory - A00" value="248">
                                    CSE140L - SPR18
                                </option>, <option title="CSE 110 - SPR18 - Software Engineering" value="228">
                                    CSE 110 - SPR18
                                </option>, <option value="-1">View Inactive</option>]


In [41]:
# remote irrelvant courses that are disabled
for i in courses:
    if i['value'] != '-1':
        currentCourses.append(i.text)

In [42]:
print(currentCourses)
#format
for i in range(len(currentCourses)):
    currentCourses[i] = currentCourses[i].replace('\n', '').strip()
#print courses on autograder
print(currentCourses)


['\n                                    CSE140L - SPR18\n                                ', '\n                                    CSE 110 - SPR18\n                                ']
['CSE140L - SPR18', 'CSE 110 - SPR18']


In [49]:
queue = souppy.find('div', {'class': 'queue-listing tab-content'})
print(queue)
print(queue.find('ul'))
temp = souppy.find_all('ul')
print(temp)

<div class="queue-listing tab-content">
<div class="tab-pane active" id="pending-tab">
</div>
<div class="tab-pane" id="tutors-tab">
</div>
<div class="tab-pane" id="history-tab">
</div>
</div>
None
[<ul class="nav navbar-nav">
<li class="dropdown">
<div style="padding: 8px 0">
<form accept-charset="utf-8" action="/basecontroller/redirectcourse" enctype="application/x-www-form-urlencoded" method="post"><input name="authenticityToken" type="hidden" value="e54ba8f5f5d725405e563be44a6cc2eb15ba5f2f"/>
<select class="form-control" id="course-select" name="courseId" style="margin: 0;">
<option selected="" title="CSE140L - SPR18 - Digital Systems Laboratory - A00" value="248">
                                    CSE140L - SPR18
                                </option>
<option title="CSE 110 - SPR18 - Software Engineering" value="228">
                                    CSE 110 - SPR18
                                </option>
<option value="-1">View Inactive</option>
</select>
<input name="